In [ ]:
# Importing Packages

import os
import pandas as pd
from datetime import datetime

### 1) Specifying and Customizing the Files Directory

In [ ]:
# Specify the directory of where the files are available
directory = input("Enter directory of the existing files:")
directory = directory.replace("\\" , "/")

# Changing Directory
wd = os.chdir(directory) 

# What are the files that is available within the directory
print("List of Available Files within Directory")
files = os.listdir(wd) 
files

### 2) Reading the Data from the Excel files and do some Transformation

In [ ]:
# Create An empty DF
df_total = pd.DataFrame()

# User Input Variable to be added into the Data
Base_Year = int(input("Enter Base Year:"))
Period = input("Enter Period: (Mid Year or End Year)")
Data_Source = input("Enter Data Source: (Open-Source or Internal)")

# Define the Time Boundary of the specified Base Year
if Period == "Year End":
    Time_Boundary = str(Base_Year + 1) + "-01-01"
else:
    Time_Boundary = str(Base_Year) + "-07-01"

# Convert into Datetime format
Time_Boundary = datetime.strptime(Time_Boundary, '%Y-%m-%d')

# Checking the Time and it's data type
print(type(Time_Boundary))
print(Time_Boundary)


# Loop through the list of files and load them into a DataFrame
for file in files: 
    if file.startswith('~$'): # Ignore files that are currently opened
        continue 
    if file.endswith('.xlsx'): # Only read Excel files (without Mqcro)
        print("Reading File: " + file)
        excel_file = pd.ExcelFile(file)
        sheets = excel_file.sheet_names
        df_sheets = pd.DataFrame()
        
        # Loop Through the sheets of each file
        for sheet in sheets:
            dftemp = pd.DataFrame()
            new = sheet.split('-')
            if not len(new) == 2: # Only read Sheets that are formatted accordingly
                continue
            else: 
                df = excel_file.parse(sheet_name = sheet,skiprows=6, usecols="C:J")
                
                df.columns = ['Date', 'Motorbikes', 'Commuters', 'Buses', 'Cars', 'Bikes', 'Walk', 'NA']
                for i in range(len(df)): # Looping through each Row
                    dfi = df.iloc[[i],]
                    if dfi['Date'].dropna().empty == True : # Stop Looping if row reached end (No Date Data)
                        break
                    elif (dfi['Date'] < Time_Boundary).bool() == True: # Only read Data within the time boundary
                        continue
                    else:
                        dftemp = dftemp.append(dfi)
                dftemp['District'] = new[1]
                
                df_sheets = df_sheets.append(dftemp)
                
        dfinal = pd.DataFrame()
        # Data Transformation, Unpivot and Pivot
        for column in df_sheets[['Motorbikes', 'Commuters', 'Buses', 'Cars', 'Bikes', 'Walk', 'NA']]: #Loop each column
            #Pivot each column
            dfreshape = df_sheets.pivot(index='Date', columns='District')[column].reset_index()
            dfreshape.columns.name = None
            dfreshape = dfreshape.fillna(0)
           
            #Unpivot 
            dfreshape = dfreshape.melt(id_vars=['Date'], var_name='District', value_name=column)
            dfinal['Date'] = dfreshape['Date']
            dfinal[column] = dfreshape[column]
            
        dfinal['Region'] = new[0]
        dfinal['District'] = dfreshape['District']
        dfinal['Period'] = Period
        dfinal['Base_Year'] = Base_Year
        dfinal['Data_Source'] = Data_Source
        
        # Final Reshaped DataFrame
        df_total = df_total.append(dfinal)

print("Data Succesfully Extracted into a Dataframe!")

In [ ]:
# Check if there are NA Values within the Date Column
df_total['Date'].isnull().sum()

In [ ]:
# Fill Missing Values with 0
df_total = df_total.fillna(0)

In [ ]:
# Get the summary of the DF
df_total.info()
df_total

### 3) Connecting Into The Database

In [ ]:
# There are multiple packages to connect Python into a Database, in this case, Im using pyodbc
import pyodbc

# Specifiy Connection
conn = pyodbc.connect('Driver={MyDrivertype};'
                      'Server=Myserver;'
                      'Database=Mydatabase;'
                      'UID=MyUID;'
                      'PWD=MyPWD')

# Create Cursor to run SQL command
cursor = conn.cursor()
print("Connection Successful!")

### 4) Loading The Data into the Database

In [ ]:
for row in df_total.itertuples():
    cursor.execute("""
                INSERT INTO MyTable (
                Date, 
                Motorbikes, 
                Commuters,
                Buses,
                Cars,
                Bikes,
                Walks,
                N/A,
                Region,
                District,
                Period,
                Base_Year,
                Data_Source)
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)
                """,
                row.Date, 
                row.Motorbikes,
                row.Commuters,
                row.Buses,
                row.Cars,
                row.Bikes,
                row.Walks,
                row.NA,
                row.Region,
                row.District,
                row.Period,
                row.Base_Year,
                row.Data_Source
                )
conn.commit()
print("Data Successfully Inserted Into The Database")

In [ ]:
# Check to See Inserted Data

sql_query = pd.read_sql_query('SELECT * FROM MyTable WHERE Base_Year = (?) AND Period = (?)',conn, params = (Base_Year, Period))
print(sql_query)

In [ ]:
# Close Connection after finish
conn.close()
print("Connection Closed!")